# 방화벽 관리

> 리눅스 방화벽
- 외부의 네트워크에서 내부의 시스템으로 접근하는 네트워크 패킷을 차단하는 서비스
- 규칙을 이용해 접근을 허용하거나 차단할 수 있음.

<img src="https://access.redhat.com/webassets/avalon/d/Red_Hat_Enterprise_Linux-7-Security_Guide-ko-KR/images/eee9192950e07b21f5c95b3ced63ae09/RHEL_Security-Guide_453350_0717_ECE_firewalld-comparison-rhel7.png" width=600 height=300>

- Netfilter(Kernel) <- Iptables(명령) <-Iptables(service)\
-                  <- Firewalld(service) <- firewall-cmd <- firewall-config

리눅스에서 제공하는 방화벽은 Netfilter에 의해서 적용됩니다.\
Netfilter는 시스템에 접근하는 네트워크 패킷을 시스템 내부로 전달할지 아니면 폐기할지 결정하는 커널 모듈입니다.\
사용자들이 Netfilter를 사용하여 네트워크 접근을 직접 제어하지 않고, 서비스 관리 도구를 이용하여 제어합니다.

리눅스에서 기본적으로 제공하는 방화벽 서비스는 두 가지가 있습니다.\
하나는 오랫동안 사용해온 iptables가 있고 다른 하나는 firewalld입니다.

### iptables
- Netfilter를 제어하는 도구
- systemd와 함께 출시된 firewalld가 최신 리눅스의 기본 방화벽 서비스로 제공되고 있어서 firewalld를 기본적으로 사용하고 있습니다.
- 단점
    - iptables는 방화벽 규칙을 변경할 때마다 서비스를 중지 후 재시작해야 합니다. 따라서 네트워크 변화가 수시로 발생하는 환경에서 iptables로 방화벽을 관리하는 것은 번거로울 뿐 아니라 

### firewalld
- 기존의 iptables의 한계와 단점을 보완하는 방화벽 서비스
- 동적으로 방화벽 설정을 변경할 수 있기 때문에 수시로 변하는 네트워크 변화에 대해 제한 없이 대응할 수 있다는 장점이 있습니다.
- firewalld와 iptables를 함께 사용하면 충돌이 발생합니다.
- 특징
    - XML파일형태로 보관
    - Runtime(실행중) 및 Permanent(영구) 설정
    - 사전에 정의된 영역(Pre-defined Zone)
    - 사전에 정의된 서비스(Pre-defined Service)
    - D-Bus 사용
- Runtime : 시스템 운영중인 상태일 때 반영
- Permanent : 서비스가 재시작 되었을떄도 반영. XML파일

Pre-defined Zone

|영역|설명|
|--|--|
|block(변경불가)|- 모든 패킷 거부(reject) </br> - 내부에서 외부로의 반환 패킷은 허용|
|dmz| - 내부로 들어오는 패킷 거부(reject) </br> - 외부로의 연결, ssh 서비스 허용|
|drop(변경불가)|- 내부로 들어오는 모든 패킷 폐기(drop) </br> - ICMP 에러도 폐기 </br> 외부로의 연결만 허용|
|external|- 내부로 들어오는 모든 패킷 거부(reject) </br> - 외부로의 연결, ssh 서비스 허용 </br> - IP Masquerading(마스커레이딩) 활성화|
|home|- 내부로 들어오는 패킷 거부(reject) </br> - dhcpv6, ipp-client, mdns, samba-client, ssh허용|
|Public|- Default Zone </br> - 시스템에 인터페이스가 추가되면 인터페이스 규칙이 추가됨. </br> - 내부로 들어오는 패킷 거부(reject) </br> - ssh, dhcpv6-client 허용|
|trusted(변경불가)|- 모든 패킷 허용|
|work|- 내부로 들어오는 패킷 거부(reject) </br> - dhcpv6-client, ipp-client, ssh 허용|

```
실습 명령어

# httpd 상태 확인
systemd status httpd

# 설치
dnf -y install httpd

systemd status httpd
systemd start httpd
systemd enable httpd --now


# firewlld
systemd start firewalld
firewall-config

firewall-cmd --state
firewall-cmd --get-default-zone
firewall-cmd --set-default-zone
firewall-cmd --set-default-zone=home
firewall-cmd --get-zones
firewall-cmd --get-services
firewall-cmd --get-active-zones
firewall-cmd --set-default-zone=public
firewall-cmd --get-active-zones
firewall-cmd --list
firewall-cmd --list-all
firewall-cmd --list-services
firewall-cmd --list-all | less
firewall-cmd --add-source={ipaddress}/24 --permanent
firewall-cmd --list-source
firewall-cmd --list-sources
firewall-cmd --list-sources --permanent
firewall-cmd --reload
firewall-cmd --list-sources
firewall-cmd --remove-source={ipaddress}/24 --permanet
firewall-cmd --reload
firewall-cmd --add-service=https --permanet
firewall-cmd --reload
firewall-cmd --list-services
firewall-cmd --add-port=23/tcp --permanent
firewall-cmd --add-port=24/udp --permanent
firewall-cmd --reload
firewall-cmd --list-ports
```